In [ ]:
import opendatasets as od
import pandas
 
od.download("https://www.kaggle.com/competitions/feather-in-focus/data")

In [1]:
#https://www.kaggle.com/code/chekoduadarsh/starters-guide-convolutional-xgboost/notebook
#https://www.kaggle.com/code/pedrolucasbritodes/bird-image-classification-cnn-89-accuracy/notebook
#https://stats.stackexchange.com/questions/404809/is-it-advisable-to-use-output-from-a-ml-model-as-a-feature-in-another-ml-model

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

#how many epchos still revert
patience1 = 3

base_learning_rate = 0.001 
#number before fine_tuning
epchos1 =15
#number after fine_tuning
number_epcho2 = 20
number_class = 200
#needs to be the same number of classes
output_Neurons = 200
Neurons = 512
batch_size = 32
drop_out_rate = 0.3
image_shape = (224,224)
image_shape_full = (224,224,3)
current_directory = "feather-in-focus/"
#current_directory = "feather-in-focus/"
blur_number = 3

2023-12-13 13:50:44.631836: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 13:50:44.823422: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
#loading files 
class_names2 = np.load('{}class_names.npy'.format(current_directory), allow_pickle=True).item()

#load all files because why not
test_image_path = pd.read_csv('{}test_images_path.csv'.format(current_directory))
test_image_path['total_path'] = current_directory + "/test_images" + test_image_path['image_path'] 
test_image_path['total_path'] = test_image_path['total_path'].str.replace("\\", "/" )
test_image = pd.read_csv('{}test_images_sample.csv'.format(current_directory))


#importing training data 
train_image = pd.read_csv('{}train_images.csv'.format(current_directory))
train_image['total_path'] = current_directory + "train_images" + train_image['image_path']
train_image['total_path'] = train_image['total_path'].str.replace("\\", "/" )
train_image['short_name'] = train_image['image_path'].str.rsplit('/', n=1).str[-1]

#train_image['label'] = train_image['label'].apply(str)
train_image.head()

#importing bird data 
bird_data = pd.read_csv('{}output_birds.csv'.format(current_directory))


bird_data2 = train_image.merge(bird_data, how = 'left', left_on = 'label', right_on = 'bird_number_x')
bird_data2['label'] = bird_data2['label'].apply(str)


M1 = 'Tarsus.Length'
M2 = 'Hand-Wing.Index'
M3 = 'Beak.Depth'
c1 = 'Primary.Lifestyle'

bird_data2 = bird_data2[['image_path','short_name', 'label', 'total_path', 'bird_class', M1,
                        M2, M3, c1]]
from sklearn.preprocessing import LabelEncoder

#label_encoder = LabelEncoder()
#ird_data2['encoded_labels'] = label_encoder.fit_transform(bird_data2[c1])
bird_data2[c1] =  pd.factorize(bird_data2[c1])[0] +1
# Convert encoded labels to one-hot encoded format
set(bird_data2[c1].tolist())

/scratch-local/scur2079.4639786/ipykernel_3936187/3679845827.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  test_image_path['total_path'] = test_image_path['total_path'].str.replace("\\", "/" )
/scratch-local/scur2079.4639786/ipykernel_3936187/3679845827.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train_image['total_path'] = train_image['total_path'].str.replace("\\", "/" )


{1, 2, 3, 4, 5}

In [3]:
Wing_min = bird_data2[M1].min()
Wing_max = bird_data2[M1].max()
bird_data2[M1] =  (bird_data2[M1] - Wing_min) / (Wing_max - Wing_min)

#Wing_mean = bird_data2['Wing.Length'].mean()
#Wing_std = bird_data2['Wing.Length'].std()
#bird_data2['Wing.Length'] =  (bird_data2['Wing.Length'] - Wing_mean) / (Wing_std)


tail_min = bird_data2[M2].min()
tail_max = bird_data2[M2].max()
#tail_mean = bird_data2['Tail.Length'].mean()
#tail_std = bird_data2['Tail.Length'].std()

#bird_data2['Tail.Length'] = (bird_data2['Tail.Length'] - tail_mean ) / tail_std 

bird_data2[M2] = (bird_data2[M2] - tail_min ) / (tail_max - tail_min) 
mass_min = bird_data2[M3].min()
mass_max = bird_data2[M3].max()
#mass_mean = bird_data2['Beak.Width'].mean()
#mass_std = bird_data2['Beak.Width'].std()
#bird_data2['Beak.Width'] = (bird_data2['Beak.Width'] - mass_mean) / mass_std
bird_data2[M3] = (bird_data2[M3] - mass_min) / (mass_max - mass_min)

import pandas as pd

# Check for missing or null values in any column
if bird_data2[[M1,M2, M3]].isnull().values.any():
    print("There are missing or null values in the DataFrame.")
    # Return True or perform necessary actions
else:
    print("There are no missing or null values in the DataFrame.")
    # Return False or proceed accordingly




There are no missing or null values in the DataFrame.


In [4]:
# we need to add a k-folds, i have stratified so it works a bit better
train, valid = train_test_split(bird_data2, 
                    test_size=0.2,
                    #stratify = train_image['label'],
                    random_state=420)


In [5]:

#valid = pd.read_csv('valid_csv.csv')
#train = pd.read_csv('train_csv.csv')
#valid['label'] = valid['label'].astype(str)
#train['label'] = train['label'].astype(str)
bird_data2

,image_path,short_name,label,total_path,bird_class,Tarsus.Length,Hand-Wing.Index,Beak.Depth,Primary.Lifestyle
0,/train_images/1.jpg,1.jpg,1,feather-in-focus/train_images/train_images/1.jpg,Black footed Albatross,0.673810,0.929712,0.613924,1
1,/train_images/2.jpg,2.jpg,1,feather-in-focus/train_images/train_images/2.jpg,Black footed Albatross,0.673810,0.929712,0.613924,1
2,/train_images/3.jpg,3.jpg,1,feather-in-focus/train_images/train_images/3.jpg,Black footed Albatross,0.673810,0.929712,0.613924,1
3,/train_images/4.jpg,4.jpg,1,feather-in-focus/train_images/train_images/4.jpg,Black footed Albatross,0.673810,0.929712,0.613924,1
4,/train_images/5.jpg,5.jpg,1,feather-in-focus/train_images/train_images/5.jpg,Black footed Albatross,0.673810,0.929712,0.613924,1
...,...,...,...,...,...,...,...,...,...
3921,/train_images/3922.jpg,3922.jpg,200,feather-in-focus/train_images/train_images/392...,Common Yellowthroat,0.140476,0.153355,0.033755,4
3922,/train_images/3923.jpg,3923.jpg,200,feather-in-focus/train_images/train_images/392...,Common Yellowthroat,0.140476,0.153355,0.033755,4
3923,/train_images/3924.jpg,3924.jpg,200,feather-in-focus/train_images/train_images/392...,Common Yellowthroat,0.140476,0.153355,0.033755,4
3924,/train_images/3925.jpg,3925.jpg,200,feather-in-focus/train_images/train_images/392...,Common Yellowthroat,0.140476,0.153355,0.033755,4


In [6]:
#deleting images if they are there 

folder_path = "custom_aug"  # Replace 'path_to_folder' with the actual path to your folder

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    # Check if the file is an image (you might want to refine this check based on your specific file types)
    if os.path.isfile(file_path) and (filename.endswith('.jpg') or filename.endswith('.png')):
        os.remove(file_path)
        #print(f"Deleted: {file_path}")

In [ ]:
import cv2
from PIL import Image, ImageFilter
import pandas as pd
import numpy as np

def custom_blur(dataframe, label, name, folder, blur_number):
    img = Image.open(dataframe['total_path']).filter(ImageFilter.GaussianBlur(blur_number))
    img.save(f"{folder}/blur_{dataframe[name]}")

    return {
        "image_path": f"{folder}/blur_{dataframe[name]}",
        M1: dataframe[M1],
        M2: dataframe[M2],
        M3: dataframe[M3],
        c1: dataframe[c1],
        "total_path": f"{folder}/blur_{dataframe[name]}",
        "short_name": f"blur_{dataframe[name]}",
        "label": dataframe[label],
    }


def custom_grey_scale(dataframe, label, name, location, folder):
    img = Image.open(dataframe[location]).convert('L')
    img.save(f"{folder}/gs_{dataframe[name]}")

    return {
        "image_path": f"{folder}/gs_{dataframe[name]}",
        M1: dataframe[M1],
        M2: dataframe[M2],
        M3: dataframe[M3],
        c1: dataframe[c1],
        "total_path": f"{folder}/gs_{dataframe[name]}",
        "short_name": f"gs_{dataframe[name]}",
        "label": dataframe[label],
    }


def custom_getPerspectiveTransform(dataframe, label, name, folder):
    input_image = cv2.imread(dataframe['total_path'])
    height, width = input_image.shape[:2]
    
    objPoints = np.float32([[0, 0], 
                            [int(np.random.uniform(0.75, 0.85) * width), 0], 
                            [0, int(np.random.uniform(0.85, 0.95) * height)], 
                            [int(np.random.uniform(0.75, 0.85) * width), int(np.random.uniform(0.65, 0.75) * height)]])
    
    imgPts = np.float32([
        [np.random.uniform(0.10, 0.20) * width, np.random.uniform(0.20, 0.30) * height],
        [np.random.uniform(0.75, 0.85) * width, np.random.uniform(0.15, 0.25) * height],
        [np.random.uniform(0.05, 0.15) * width, np.random.uniform(0.65, 0.75) * height],
        [np.random.uniform(0.85, 0.95) * width, np.random.uniform(0.65, 0.75) * height]
    ])
    
    matrix = cv2.getPerspectiveTransform(imgPts, objPoints)
    result = cv2.warpPerspective(input_image, matrix, (width, height))
    
    cv2.imwrite(f"{folder}/persp_{dataframe[name]}", result)

    return {
        "image_path": f"{folder}/persp_{dataframe[name]}",
        M1: dataframe[M1],
        M2: dataframe[M2],
        M3: dataframe[M3],
        c1: dataframe[c1],
        "total_path": f"{folder}/persp_{dataframe[name]}",
        "short_name": f"persp_{dataframe[name]}",
        "label": dataframe[label],
    }

loc = "custom_aug" 
blur_number = 5  # Specify your blur number

train_augmented_data = []
valid_augmented_data = []

for index, row in train.iterrows():
    train_augmented_data.append(custom_getPerspectiveTransform(row, 'label', 'short_name', loc))
    train_augmented_data.append(custom_blur(row, 'label', 'short_name', loc, blur_number))
    train_augmented_data.append(custom_grey_scale(row, 'label', 'short_name','total_path', loc))
    

for index, row in valid.iterrows():
    valid_augmented_data.append(custom_getPerspectiveTransform(row, 'label', 'short_name', loc))
    valid_augmented_data.append(custom_blur(row, 'label', 'short_name', loc, blur_number))
    valid_augmented_data.append(custom_grey_scale(row, 'label', 'short_name','total_path', loc))

# Create augmented DataFrames
persp_train_dataframe = pd.DataFrame(train_augmented_data)
persp_valid_dataframe = pd.DataFrame(valid_augmented_data)

# Concatenate augmented DataFrames with original DataFrames
train = pd.concat([train, persp_train_dataframe], ignore_index=True, sort=False)
valid = pd.concat([valid, persp_valid_dataframe], ignore_index=True, sort=False)


In [ ]:
import pandas as pd
#valid = pd.read_csv('valid_csv.csv')
#train = pd.read_csv('train_csv.csv')
#valid['label'] = valid['label'].astype(str)
#train['label'] = train['label'].astype(str)
#train

In [ ]:
# this creates the type of augmentation that is done! 

datagen = ImageDataGenerator(
    #rotation_range=15,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range = 15,
    horizontal_flip = True,
    zoom_range = 0.20)
#look at aug from group chat 

#okay this method seems WAY better. only ever use aug images
train_generator = datagen.flow_from_dataframe(
    dataframe=train,
    x_col='total_path',
    y_col=[M1,M2,M3],
           # Assuming 'label' contains the continuous values
    color_mode='rgb',
    target_size=image_shape,
    class_mode='raw',  # Use 'raw' for continuous values instead of 'categorical'
    batch_size=batch_size  # Define your batch size
)


valid_generator = datagen.flow_from_dataframe(
    dataframe=valid,
    x_col='total_path',
    y_col=[M1, M2, M3],
    color_mode='rgb',
    target_size=image_shape,
    class_mode='raw',
    batch_size=batch_size
)

In [ ]:
from keras.applications import ResNet50, InceptionV3
from keras.models import Model
from keras.layers import Dense, BatchNormalization, Dropout, GlobalAveragePooling2D, Input
import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
import tensorflow as tf

dr = 0.4
IM_WIDTH, IM_HEIGHT = 224, 224

class MultiOutputBird():
    def __init__(self):
        self.base_model_resnet = ResNet50(
            include_top=False,
            input_shape=(IM_HEIGHT, IM_WIDTH, 3),
            pooling='avg',
            weights='imagenet'
        )
        #self.base_model_resnet.trainable = False

        self.base_model_inception = InceptionV3(
            include_top=False,
            input_shape=(IM_HEIGHT, IM_WIDTH, 3),
            pooling='avg',
            weights='imagenet'
        )
        
        for layer in self.base_model_inception.layers[:-10]:
            layer.trainable = False

        for layer in self.base_model_resnet.layers[:-10]:
            layer.trainable = False

        #self.base_model_inception.trainable = False
    def tail_length_branch(self, inputs, dropout_rate=0.3, l2_reg=0):
        # Hidden layer 1
        x = Dense(1024)(inputs)
        x = BatchNormalization()(x)
        x = tf.keras.activations.relu(x)
        x = Dropout(dropout_rate)(x)

        # Hidden layer 2
        x = Dense(512, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x)
        x = BatchNormalization()(x)
        x = tf.keras.activations.relu(x)
        x = Dropout(dropout_rate)(x)

        # Hidden layer 3
        x = Dense(512)(x)
        x = BatchNormalization()(x)
        x = tf.keras.activations.relu(x)

        # Output layer
        x = Dense(1, activation='sigmoid', name="{}_output".format(M2))(x)
        return x

    def tail_length_branch(self, inputs, dropout_rate=0.3, l2_reg=0):   
        x = Dense(1024)(inputs)
        x = BatchNormalization()(x)
        x = tf.keras.activations.relu(x)
        x = Dropout(dropout_rate)(x)

        # Hidden layer 2
        x = Dense(512, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x)
        x = BatchNormalization()(x)
        x = tf.keras.activations.relu(x)
        x = Dropout(dropout_rate)(x)

        # Hidden layer 3
        x = Dense(512)(x)
        x = BatchNormalization()(x)
        x = tf.keras.activations.relu(x)
        
        x = Dense(1, activation='sigmoid',  name='{}_output'.format(M2))(x)
        return x

    def beak_branch(self, inputs, dropout_rate=0.3, l2_reg=0):   
        x = Dense(1024)(inputs)
        x = BatchNormalization()(x)
        x = tf.keras.activations.relu(x)
        x = Dropout(dropout_rate)(x)

        # Hidden layer 2
        x = Dense(512, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x)
        x = BatchNormalization()(x)
        x = tf.keras.activations.relu(x)
        x = Dropout(dropout_rate)(x)

        # Hidden layer 3
        x = Dense(512)(x)
        x = BatchNormalization()(x)
        x = tf.keras.activations.relu(x)
        
        x = Dense(1, activation='sigmoid', name="{}_output".format(M3))(x)
        return x
    
    def wing_length_branch(self, inputs, dropout_rate=0.3, l2_reg=0):   
        x = Dense(1024)(inputs)
        x = BatchNormalization()(x)
        x = tf.keras.activations.relu(x)
        x = Dropout(dropout_rate)(x)

        # Hidden layer 2
        x = Dense(512, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x)
        x = BatchNormalization()(x)
        x = tf.keras.activations.relu(x)
        x = Dropout(dropout_rate)(x)

        # Hidden layer 3
        x = Dense(512)(x)
        x = BatchNormalization()(x)
        x = tf.keras.activations.relu(x)
        
        x = Dense(1, activation='sigmoid', name='{}_output'.format(M1))(x)
        return x
    
    #def multi_class(self, inputs, dropout_rate=0.3, l2_reg=0.02, num_classes=len(set(bird_data2[c1].tolist()))):   
        #x = Dense(1024)(inputs)
        #x = BatchNormalization()(x)
        #x = tf.keras.activations.relu(x)
        #x = Dropout(dropout_rate)(x)

        # Hidden layer 2
        #x = Dense(512, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x)
        #x = BatchNormalization()(x)
        #x = tf.keras.activations.relu(x)
       # x = Dropout(dropout_rate)(x)

        # Hidden layer 3
      #  x = Dense(512)(x)
      #  x = BatchNormalization()(x)
      #  x = tf.keras.activations.relu(x)

      #  x = Dense(num_classes, activation='softmax', name="{}_output".format(c1))(x)
      #  return x

    
    def assemble_full_model(self):
        input_resnet = Input(shape=(IM_HEIGHT, IM_WIDTH, 3))
        input_inception = Input(shape=(IM_HEIGHT, IM_WIDTH, 3))

        x_resnet = self.base_model_resnet(input_resnet)
        x_inception = self.base_model_inception(input_resnet)

        merged = tf.keras.layers.concatenate([x_resnet, x_inception])

        M2_output = self.tail_length_branch(merged)
        M3_output = self.beak_branch(merged)
        M1_output = self.wing_length_branch(merged)
        #c1_output = self.multi_class(merged)

        model = Model(inputs=[input_resnet], outputs=[M1_output, M2_output, M3_output], name="multi_output_bird_net")
        return model

# Example usage:
model_builder = MultiOutputBird()
model = model_builder.assemble_full_model()

In [ ]:
from keras.optimizers import Adam
epochs = 15
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)

model.compile(optimizer=adam_optimizer,
              loss={
                  '{}_output'.format(M1): 'mse',
                  '{}_output'.format(M2): 'mse',
                  '{}_output'.format(M3): 'mse'
                #'{}_output'.format(c1):  "categorical_crossentropy"
                  
              },
              metrics={
                  '{}_output'.format(M1): 'mae',
                  '{}_output'.format(M2): 'mae',
                  '{}_output'.format(M3): 'mae'
                 # '{}_output'.format(c1):  "accuracy"
              })



In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

early_stop = tf.keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=5, min_lr=0.0001)

In [ ]:
history_1 = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=epchos1,
    initial_epoch=epchos1,
    batch_size=32,
    callbacks=[reduce_lr, early_stop]
)

In [ ]:
history_1 = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=epchos1 + 15,
    initial_epoch=epchos1,
    batch_size=32,
    callbacks=[reduce_lr, early_stop]
)

In [ ]:
import keras
from matplotlib import pyplot as plt
plt.clf() 
plt.plot(history.history['{}_output_mae'.format(M1)])
plt.plot(history.history['val_{}_output_mae'.format(M1)])
plt.title('model accuracy {}_output'.format(M1))
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('continous_result_{}.png'.format(M1))
plt.clf() 

import keras
from matplotlib import pyplot as plt
plt.plot(history.history['{}_output_mae'.format(M2)])
plt.plot(history.history['val_{}_output_mae'.format(M2)])
plt.title('model accuracy {}'.format(M2))
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('continous_result_wing.png')
plt.clf() 


import keras
from matplotlib import pyplot as plt
plt.plot(history.history['val_{}_output_mae'.format(M3)])
plt.plot(history.history['{}_output_mae'.format(M3)])
plt.title('model accuracy {}'.format(M3))
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('continous_result_tail.png'.format(M1))
plt.clf() 

In [ ]:

current_directory = "feather-in-focus"
model.save('{}/keras_final_continous2.keras'.format(current_directory))


# image folder
folder_path = '{}/test_images/test_images/'.format(current_directory)
# path to model
model_path = '{}/keras_final_continous2.keras'.format(current_directory)
# dimensions of images
img_width, img_height = 224, 224



In [ ]:
from keras.models import load_model
import keras.utils as image
# load the trained model
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
model_path = '{}/keras_final_continous2.keras'.format(current_directory)
model = load_model(model_path)
model.compile(optimizer=adam_optimizer,
              loss={
                  '{}_output'.format(M1): 'mse',
                  '{}_output'.format(M2): 'mse',
                  '{}_output'.format(M3): 'mse'
                #'{}_output'.format(c1):  "accuracy"
                  
              },
              metrics={
                  '{}_output'.format(M1): 'mae',
                  '{}_output'.format(M2): 'mae',
                  '{}_output'.format(M3): 'mae'
                 # '{}_output'.format(c1):  "accuracy"
              })


In [ ]:
import numpy as np
from heapq import nlargest

def closest_indices_mask(array, value, num):
    closest_indices = []
    distances = []

    for i, arr_value in enumerate(array):
        distance = np.linalg.norm(np.array(arr_value) - np.array(value))
        if len(closest_indices) < num:
            closest_indices.append((i, distance))
            distances.append(distance)
        else:
            max_distance = max(distances)
            if distance < max_distance:
                max_index = distances.index(max_distance)
                closest_indices[max_index] = (i, distance)
                distances[max_index] = distance

    closest_indices.sort(key=lambda x: x[1])  # Sort by distance
    closest_indices = [index for index, _ in closest_indices]

    closest_mask = np.zeros_like(array)
    closest_mask[closest_indices] = 1
    return closest_mask[:, 0]

In [ ]:
name = list()
images = []
folder_path = 'feather-in-focus/train_images/train_images'  
#folder_path = 'feather-in-focus/test_images/test_images' 
for img_name in os.listdir(folder_path):
    img_path = os.path.join(folder_path, img_name)
    # Check if the item is a file (not a directory)
    if os.path.isfile(img_path):
        name.append(img_name)
        img = image.load_img(img_path, target_size=image_shape)
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        images.append(img)
        # Process the image further as needed
    else:
        # Skip directories
        continue
        
# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.predict(images, batch_size=10)
#classes_x=np.argmax(classes,axis=1)

In [ ]:
cols = [M1, M2, M3,c1, 'label']
idea = bird_data2[cols].sort_values(by=['label']).drop_duplicates()
cols = [M1, M2, M3]
idea  = idea[cols]
array2 = idea.to_numpy().tolist()

result = [[classes[0][i], classes[1][i], classes[2][i]] for i in range(min(len(classes[0]), len(classes[1]), len(classes[2])))]
distances = []
for arr1 in result:
    distances_to_arr1 = []
    for arr2 in array2:
        # Calculate Euclidean distance between elements in arr1 and arr2
        distance = np.linalg.norm(np.array(arr1) - np.array(arr2))
        distances_to_arr1.append(distance)
    distances.append(distances_to_arr1)

#print(distances)


In [ ]:
selected_columns = [ '{}'.format(M1), '{}'.format(M2), '{}'.format(M3), c1, 'label']

dictionary = dict()
for index, row in bird_data2[selected_columns].iterrows():
    key = (row['{}'.format(M1)], row['{}'.format(M2)], row['{}'.format(M3)])
    value = row['label']
    dictionary[key] = value


In [ ]:

cols = [M1,M2,M3,c1 ,'label']
idea = bird_data2[cols].sort_values(by=['label']).drop_duplicates()
cols = [M1,M2,M3]
idea  = idea[cols]
array2 = idea.to_numpy().tolist()
#array2


result = [[classes[0][i], classes[1][i], classes[2][i]] for i in range(min(len(classes[0]), len(classes[1]), len(classes[2])))]
distances = []
for arr1 in result:
    distances_to_arr1 = []
    for arr2 in array2:
        # Calculate Euclidean distance between elements in arr1 and arr2
        distance = np.linalg.norm(np.array(arr1) - np.array(arr2))
        distances_to_arr1.append(distance)
    distances.append(distances_to_arr1)




In [ ]:
# save numpy array as npy file
from numpy import asarray
from numpy import save
# define data
# save to npy file
save('data_lookup.npy', array2)


In [ ]:
final_df = pd.DataFrame(
    {'id': name,
     '{}_P'.format(M1): classes[0].tolist(),
     '{}_P'.format(M2): classes[1].tolist(),
     '{}_P'.format(M3): classes[2].tolist(),
     '{}_P'.format(c1): np.argmax(classes[3],axis=1),
    })
final_df['{}_P'.format(M1)] = final_df['{}_P'.format(M1)].apply(lambda x: float(x[0]))
final_df['{}_P'.format(M2)] = final_df['{}_P'.format(M2)].apply(lambda x: float(x[0]))
final_df['{}_P'.format(M3)] = final_df['{}_P'.format(M3)].apply(lambda x: float(x[0]))
#final_df['{}_P'.format(c1)] = final_df['{}_P'.format(c1)].apply(lambda x: int(x[0]))

final_df


In [ ]:
np.mean(final_df['{}_P'.format(c1)])

In [ ]:
import numpy as np
import pandas as pd

def find_closest_values(dictionary, dataframe, columns,num):
    closest_values = []

    for _, row in dataframe.iterrows():
        current_row_values = row[columns].values
        closest_keys = sorted(dictionary.keys(), key=lambda k: np.linalg.norm(np.array(k) - np.array(current_row_values)))
        closest_values.append([dictionary[key] for key in closest_keys[:num]])

    return closest_values

#closest_values = find_closest_values(dictionary, final_df3[1:], ['Wing_P', 'Beak_P', 'Tail_P'], 10)


In [ ]:
final_df['id2'] = "/train_images/"+final_df['id']

In [ ]:
final_df2 = final_df.merge(train, left_on = 'id2', right_on = 'image_path')

In [ ]:
final_df3 = final_df2[['id', '{}_P'.format(M1) , '{}_P'.format(M2), '{}_P'.format(M3), '{}'.format(M1), '{}'.format(M2), '{}'.format(M3), 'label','{}'.format(c1), '{}_P'.format(M3)]]
final_df3.corr()

In [ ]:
final_df3

In [ ]:

final_df3 = final_df2[['id', '{}_P'.format(M1), '{}_P'.format(M2), '{}_P'.format(M3), '{}'.format(M1), '{}'.format(M2), '{}'.format(M3), 'label']]
#final_df3['estimates'] = closest_values

closest_values = find_closest_values(dictionary, final_df3[1:], [ '{}_P'.format(M1), '{}_P'.format(M2), '{}_P'.format(M3)],40 )

list_of_lists = [list(inner_array) for inner_array in closest_values]

# Create a DataFrame with the converted lists
new_df = pd.DataFrame({'predictions': list_of_lists})

# Concatenate the new DataFrame with the existing DataFrame
result_df = pd.concat([final_df3, new_df], axis=1)
result_df['is_value_in_prediction'] = result_df.apply(lambda row: row['label'] in row['predictions'] if isinstance(row['predictions'], list) else False, axis=1)
#result_df['is_value_in_prediction'] = result_df.apply(lambda row: row['label'] in row['predictions'], axis=1)


# Count the number of columns with True values
#num_columns_with_true = (result_df['is_value_in_prediction'] == True).any().sum()
#len(result_df[result_df['is_value_in_prediction'] == True])
len(result_df[result_df['is_value_in_prediction'] == True]) / len(result_df[result_df['is_value_in_prediction'] == False])



In [ ]:
import numpy as np

def find_closest_values(dictionary, dataframe, columns, num):
    closest_values = []
    closest_distances = []

    for _, row in dataframe.iterrows():
        current_row_values = row[columns].values
        closest_keys = sorted(dictionary.keys(), key=lambda k: np.linalg.norm(np.array(k) - np.array(current_row_values)))
        closest_classes = [dictionary[key] for key in closest_keys[:num]]
        closest_dist = [np.linalg.norm(np.array(key) - np.array(current_row_values)) for key in closest_keys[:num]]
        
        closest_values.append(closest_classes)
        closest_distances.append(closest_dist)

    return closest_values, closest_distances
#closest_values

In [ ]:
print(model.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
idea = list()
for x in array:
    idea.append(closest_indices_mask(array2, x, 3).tolist())
indices_with_ones = [i for i, sublist in enumerate(idea) if 1 in sublist]


    

In [ ]:

final_df['id2'] = "/test_images/"+final_df['id']
final_df['label_predict'] = final_df['label_predict'].astype(int)
final_df = final_df[['label_predict','id2']]

idea = np.sort(train_image['label'].unique())
idea2 = pd.DataFrame(idea)
idea2['index1'] = idea2.index
idea2.columns = ['values', 'index1']
idea2['values'] = idea2['values'].astype(int)

final_df4 = final_df.merge(test_image_path[['image_path','id']], left_on = 'id2', right_on='image_path')

final_df4 = final_df4[['id','label_predict']]

final_df2 = final_df4.merge(idea2, how = 'left', left_on = 'label_predict', right_on = 'index1')
final_df3 = final_df2[['id','values']]
#two issues the class values arent right
final_df3.columns = ['id','label']
final_df3


In [ ]:
final_df3.to_csv('output_folder/output.csv', index=False)